In [1]:
import sys
import os
import yaml
import numpy as np
import soundfile as sf
from datetime import datetime, timedelta
from pathlib import Path
import importlib as imp

In [2]:
homedir = str(Path.home())
sentineldir = str(Path.home() / 'sentinel/')
configdir = str(Path.home() / 'sentinel' / 'configs')
configdir2 = str(Path.home() / 'acoustic_field' / 'config')
recorddir = str(Path.home() / 'Recordings/')
sys.path.append(homedir)
import acoustic_field.soundscape as sc
imp.reload(sc)

<module 'acoustic_field.soundscape' from 'C:\\Users\\Camilo\\acoustic_field\\soundscape.py'>

In [6]:
with open(configdir + '/options.yaml') as file:
        opt = yaml.load(file, Loader=yaml.FullLoader)
opt        

{'sesion': 'Rivarola',
 'logfile': '/home/pi/',
 'nchan': 2,
 'nbytes': 2,
 'pkeys': ['aci', 'bi', 'ndsi', 'aei', 'adi', 'hs', 'ht', 'sc', 'db'],
 'GSM': {'apn': '"wap.gprs.unifon.com.ar"',
  'url': 'https://script.google.com/macros/s/AKfycbyQX_xnblS1c3M712v-OItr6Se6FcfE19WsiIfW/exec',
  'timeout': 180,
  'offset_file': '/tmp/offset.tmp',
  'queue': '/tmp/queue.tmp'}}

In [5]:
with open(configdir2 + '/defaults.yaml') as file:
    par = yaml.load(file, Loader=yaml.FullLoader)
par    

{'sr': 48000,
 'windowSize': 480,
 'hipass': False,
 'Filtering': {'sr': 48000, 'order': 8, 'lowcut': 300},
 'Spectrogram': {'windowSize': 480,
  'overlap': 0,
  'sr': 48000,
  'windowType': 'hanning',
  'normalized': False,
  'logf': False},
 'Indices': {'Parameters': ['ACI',
   'ADI',
   'AEI',
   'BI',
   'NDSI',
   'HS',
   'HT',
   'H'],
  'freq_BI': [2000, 8000],
  'freq_anthro': [1000, 2000],
  'freq_bio': [2000, 11000],
  'max_freq': 10000,
  'freq_step': 1000,
  'db_threshold': -50,
  'tol': 1e-12,
  'windowSize': 480,
  'window': 60,
  'half_window': 0,
  'number_of_windows': 5}}

In [60]:
# LA PESADILLA DE LAS VENTANAS
nsamples = 300 * 48000
print(nsamples)
# ventana de analisis de parametros 
par['Indices']['window'] = 60
# ventana de analisis de fourier
par['windowSize'] = 480 # en samples 10 ms
# el tamano de la ventana de analisis de parametros medido en terminos de ventanas de fourier es
FW = int(np.floor(par['Indices']['window']*par['sr']/(par['windowSize'])))
print(FW)
# media ventana de analisis 
HW = int(np.floor(par['Indices']['window']*par['sr']/(2*par['windowSize'])))
print(HW)
par['Indices']['half_window'] = HW
NOWF = int(np.floor(nsamples / par['windowSize']))
print(NOWF)
# con overlap de 50%
par['Indices']['number_of_windows'] = int(np.floor((NOWF-HW)/HW))
print('con overlap 50%')
print(par['Indices']['number_of_windows'])
# sin overlap
par['Indices']['number_of_windows'] = int(np.floor(NOWF/FW))
print('sin overlap')
print(par['Indices']['number_of_windows'])

14400000
6000
3000
30000
con overlap 50%
9
sin overlap
5


In [ ]:
nsamples = 300 * 48000
print(nsamples)
NOWF = int(np.floor(nsamples / par['windowSize']))
if overlap:
    HW = int(np.floor(par['Indices']['window']*par['sr']/(2*par['windowSize'])))
    par['Indices']['number_of_windows'] = int(np.floor((NOWF-HW)/HW))
else:
    FW = int(np.floor(par['Indices']['window']*par['sr']/(par['windowSize'])))
    par['Indices']['number_of_windows'] = int(np.floor(NOWF/FW))


In [57]:
files = os.listdir(recorddir)
files_data = filter(lambda k: ('.pcm' in k), files)

In [59]:
nmax = 2**(opt['nbytes']*8-1)
with open('pepe.csv', "a") as fp:
    for f in files_data:
        print(f)
        dn=[int(s) for s in fname.split('.')[0].split('_')]
        tlast = datetime(dn[0],dn[1],dn[2],dn[3],dn[4],dn[5])
        dump = np.fromfile(recorddir + '/' + f, dtype='u1', count=-1)
        dump = np.delete(dump,np.arange(0,dump.size,3))
        nsamples=dump.shape[0]//(opt['nchan']*opt['nbytes'])
        data=np.reshape(dump.view(np.int16)/nmax,(nsamples,opt['nchan'])).astype('float32')
        spec = sc.spectrogram(data[:,0],**par['Spectrogram'])
        ind = sc.indices(spec,**par['Indices'])
        dur = ind['nsamples']/par['sr']
        indt = dur - ind['t']
        tlist = [tlast - timedelta(seconds=t.item()) for t in indt]
        for n,t in enumerate(tlist):
            csvline = t.strftime("%Y-%m-%dT%H:%M:%SZ") + ","
            csvline += ",".join([str(np.around(ind[k][0,n],decimals=3)) for k in opt['pkeys']]) + "\n"
            fp.write(csvline)


2021_05_18_19_36_15.pcm
(5, 1, 241, 6000)
(5, 1, 2880000)
(6000,)
2021_05_18_19_37_15.pcm
(5, 1, 241, 6000)
(5, 1, 2880000)
(6000,)
2021_05_18_19_38_15.pcm
(5, 1, 241, 6000)
(5, 1, 2880000)
(6000,)
2021_05_18_19_39_15.pcm
(5, 1, 241, 6000)
(5, 1, 2880000)
(6000,)
2021_05_18_19_40_15.pcm
(5, 1, 241, 6000)
(5, 1, 2880000)
(6000,)
2021_05_18_19_41_15.pcm
(5, 1, 241, 6000)
(5, 1, 2880000)
(6000,)
2021_05_18_19_42_15.pcm
(5, 1, 241, 6000)
(5, 1, 2880000)
(6000,)
2021_05_18_19_43_15.pcm
(5, 1, 241, 6000)
(5, 1, 2880000)
(6000,)
2021_05_18_19_44_15.pcm
(5, 1, 241, 6000)
(5, 1, 2880000)
(6000,)
2021_05_18_19_45_15.pcm
(5, 1, 241, 6000)
(5, 1, 2880000)
(6000,)
